# 이미지 추론

In [ ]:
# home directory 설정
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
# # Pip install method (recommended)

# !pip install ultralytics==8.0.28
# # !pip install ultralytics --upgrade

# from IPython import display
# display.clear_output()

# import ultralytics
# ultralytics.checks()

In [ ]:
from ultralytics import YOLO
from IPython.display import display, Image
import numpy as np

In [ ]:
# 저장된 모델 불러오기

# model=YOLO('yolov8n.pt')
model=YOLO('yolov8s-seg.pt')

path_best_weights = HOME+"/runs/segment/train/weights/best.pt"
model = YOLO(path_best_weights)

In [ ]:
import cv2
from PIL import Image

# # # tensor 값만 출력
# # 이미지 파일 경로 설정
# image_path = HOME + "/datasets/instance-segmentation-plastic-3/test/images/PE_088_4_jpg.rf.db48b262b02eb7e26519ee128a026f90.jpg"

# # 이미지 열기
# img = Image.open(image_path)

# # 이미지를 YOLOv8에 입력으로 전달하여 추론 수행
# results = model(img)

# # 결과 출력
# print(results)

# 모델 추론 후 저장
model.predict('./sample_image/', save=True, conf=0.5)

In [ ]:
HOME

In [ ]:
# 결과 추론된 이미지 저장

# !yolo task=segment mode=predict model={HOME}/runs/segment/train/weights/best.pt conf=0.25 source={dataset.location}/test/images save=True
!yolo task=segment mode=predict model={HOME}/runs/segment/train/weights/best.pt conf=0.25 source={HOME}/source save=True

# Roboflow API를 활용한 추론

In [ ]:
# home directory 설정
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
# roboflow 모델 api 호출하여 추론

from roboflow import Roboflow

rf = Roboflow(api_key="JGgiIMkKOOk1lVem820O")
project = rf.workspace().project("instance-segmentation-plastic")
model = project.version(5).model

image_path = HOME + "/datasets/instance-segmentation-plastic-5/test/images/PE_088_4_jpg.rf.db48b262b02eb7e26519ee128a026f90.jpg"

# 모델 추론 후 JSON형식으로 결과 확인
print(model.predict(image_path).json())

# # 모델 host로 추론
# print(model.predict("URL_OF_YOUR_IMAGE").json())

# 모델이 예측한 이미지 저장
model.predict(image_path).save("prediction.jpg")

# video 영상 추론

In [27]:
import cv2
from ultralytics import YOLO

# model_path = './runs/segment/train/weights/best.pt'
model_path = './runs/detect/train2/weights/best.pt'
model = YOLO(model_path)

# True : 동영상 파일 실행 | False : 웹캠 실행
use_video_file = True  
# use_video_file = False  

if use_video_file:
    video_path = "./video/01.mp4"
    vc1 = cv2.VideoCapture(video_path)

else : 
    vc1 = cv2.VideoCapture(0)

while vc1.isOpened() : 
    success, frame = vc1.read()

    if success : 
        result = model(frame, conf=0.3) # conf는 0.7이상으로 줘야 캔이나 다른 사물을 플라스틱으로 인식하지 않는다.

        annotated_frame = result[0].plot()
        detections = result[0].boxes

        if len(detections) > 0:
            # print("detections:", detections)
            
            # 좌표값 Print
            for box, label_cls in zip(detections.xyxy, detections.cls):

                print("box:", box) # 바운딩 box xyxy좌표 출력

                # center 좌표값
                center_x = int((box[0] + box[2]) / 2)
                center_y = int((box[1] + box[3]) / 2)
                # print("center x:", center_x, "center y:", center_y)

                print("Label:", label_cls) # label 클래스 tensor값 출력
                print("Label name:", model.names[int(label_cls)]) # label 클래스 tensor값 출력

                # center point를 점으로 표출
                cv2.circle(annotated_frame, (center_x, center_y), 5, (0, 255, 0), -1)
                
                # center point의 (x, y) 좌표도 같이 입력하여 표출
                center_text = f"({center_x}, {center_y})"
                name_text = f"{model.names[int(label_cls)]}"
                cv2.putText(annotated_frame, center_text, (center_x + 10, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.putText(annotated_frame, name_text, (center_x + 10, center_y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow("YOLOv8 Inference", annotated_frame)


        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else : 
        break


vc1.release()
cv2.destroyAllWindows()


0: 384x640 1 PS, 110.5ms
Speed: 1.0ms preprocess, 110.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
box: tensor([534.6565, 480.0986, 566.0659, 520.3749])
Label: tensor(3.)
Label name: PS

0: 384x640 1 PS, 121.0ms
Speed: 2.0ms preprocess, 121.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
box: tensor([534.6565, 480.0986, 566.0659, 520.3749])
Label: tensor(3.)
Label name: PS

0: 384x640 1 PS, 104.5ms
Speed: 1.0ms preprocess, 104.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
box: tensor([537.0594, 496.9736, 570.5792, 537.7902])
Label: tensor(3.)
Label name: PS

0: 384x640 1 PS, 112.5ms
Speed: 1.0ms preprocess, 112.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)
box: tensor([537.1833, 496.5038, 570.3430, 536.7228])
Label: tensor(3.)
Label name: PS

0: 384x640 (no detections), 111.5ms
Speed: 1.0ms preprocess, 111.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detec

In [39]:
import os
import cv2
from ultralytics import YOLO
from datetime import datetime

def initialize_yolo_model(model_path='./runs/detect/train2/weights/best.pt'):
    model = YOLO(model_path)
    return model

def open_video_capture(use_video_file=True, video_path="./video/01.mp4"):
    if use_video_file:
        return cv2.VideoCapture(video_path)
    else:
        return cv2.VideoCapture(0)

def log_detection(log_file, box, label_cls, model_names):
    center_x = int((box[0] + box[2]) / 2)
    center_y = int((box[1] + box[3]) / 2)
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    log_file.write(f"{current_time} - 바운딩 박스: {box}, 라벨: {model_names[int(label_cls)]}, 중심: ({center_x}, {center_y})\n")

def display_annotated_frame(frame, result, model):
    annotated_frame = result[0].plot()
    detections = result[0].boxes

    if len(detections) > 0:
        for box, label_cls in zip(detections.xyxy, detections.cls):
            print("바운딩 박스:", box)
            center_x = int((box[0] + box[2]) / 2)
            center_y = int((box[1] + box[3]) / 2)
            print("라벨:", label_cls)
            print("라벨 이름:", model.names[int(label_cls)])

            cv2.circle(annotated_frame, (center_x, center_y), 5, (0, 255, 0), -1)
            center_text = f"({center_x}, {center_y})"
            name_text = f"{model.names[int(label_cls)]}"
            cv2.putText(annotated_frame, center_text, (center_x + 10, center_y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            cv2.putText(annotated_frame, name_text, (center_x + 10, center_y - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("YOLOv8 추론", annotated_frame)
    return cv2.waitKey(1) & 0xFF == ord("q")

def release_resources(vc):
    vc.release()
    cv2.destroyAllWindows()

# Main function

# use_video_file값이 True : 동영상 파일 실행 | False : 웹캠 실행  
def main(use_video_file=True, video_path="./video/01.mp4"):
    model = initialize_yolo_model()
    vc = open_video_capture(use_video_file, video_path)

    log_response = input("감지 정보를 로그에 저장하시겠습니까? (y/n): ")
    log_detections = log_response.lower() == 'y'

    if log_detections:
        log_file_path = "detection_logs.txt"
        
        if not os.path.exists(log_file_path):
            # log파일 없을 경우 생성 
            with open(log_file_path, "w"):
                pass

        log_file = open(log_file_path, "a")

    while vc.isOpened():
        success, frame = vc.read()

        if success:
            result = model(frame, conf=0.3)

            if log_detections:
                for box, label_cls in zip(result[0].boxes.xyxy, result[0].boxes.cls):
                    log_detection(log_file, box, label_cls, model.names)

            if display_annotated_frame(frame, result, model):
                break
        else:
            break

    if log_detections:
        log_file.close()
        print(f"감지 정보가 {log_file_path}에 저장되었습니다.")

    release_resources(vc)

# 함수 호출 (기본 설정: 동영상 파일 사용, video_path는 "./video/01.mp4")
if __name__=="__main__":
    main()



0: 480x640 (no detections), 139.5ms
Speed: 1.0ms preprocess, 139.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 143.7ms
Speed: 1.0ms preprocess, 143.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 135.5ms
Speed: 1.0ms preprocess, 135.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 144.5ms
Speed: 1.0ms preprocess, 144.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 142.0ms
Speed: 1.0ms preprocess, 142.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 145.5ms
Speed: 2.0ms preprocess, 145.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 140.5ms
Speed: 1.0ms preprocess, 140.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 141.0ms
Speed: 1.0ms prepr